In [17]:
from langchain_community.vectorstores import ElasticsearchStore
from langchain_community.embeddings import HuggingFaceEmbeddings

import os

# Embedding model configuration
EMBEDDING_MODEL_PATH = "F:\\chat_test\\ai_models\\intfloat\\multilingual-e5-large-instruct"
EMBEDDING_MODEL_CACHE_DIR = "F:\\chat_test\\ai_models\intfloat\\multilingual-e5-large-instruct"

# Elasticsearch configuration
ELASTICSEARCH_URL = "http://127.0.0.1:9200"


embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_PATH,
    cache_folder=EMBEDDING_MODEL_CACHE_DIR,
    model_kwargs={"device": "cpu"},                    # GPU 사용 안함
    encode_kwargs={"normalize_embeddings": True}       # 임베딩 정규화 (권장)
)

elastic_vector_search = ElasticsearchStore(
    index_name="documents",
    embedding=embeddings,
    es_url=ELASTICSEARCH_URL,
    es_user="elastic",  # 보안이 비활성화된 경우
    es_password="elastic",  # 보안이 비활성화된 경우
)

In [19]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="개인정보 보호법에 따라 개인정보 수집 시에는 반드시 이용자의 동의를 받아야 합니다.",
    metadata={"source": "법률"},
)

document_2 = Document(
    page_content="개인정보 유출 시에는 지체 없이 해당 사실을 이용자에게 알리고 대통령령으로 정하는 바에 따라 조치한 결과를 이용자에게 알려야 합니다.",
    metadata={"source": "법률"},
)

document_3 = Document(
    page_content="개인정보 보호책임자는 개인정보 처리에 관한 법률 및 이 법의 준비와 집행을 맡는 이사 또는 이와 동등한 임원으로 지정하여야 합니다.",
    metadata={"source": "법률"},
)

document_4 = Document(
    page_content="개인정보의 수집과 이용 목적이 달성되면 지체 없이 해당 개인정보를 파기해야 합니다.",
    metadata={"source": "법률"},
)

document_5 = Document(
    page_content="개인정보의 안전성 확보를 위해 암호화, 접근제한, 접속이력 관리 등의 기술적 조치를 취해야 합니다.",
    metadata={"source": "법률"},
)

document_6 = Document(
    page_content="개인정보 보호법 위반 시 5년 이하의 징역 또는 5천만원 이하의 벌금이 부과될 수 있습니다.",
    metadata={"source": "법률"},
)

document_7 = Document(
    page_content="개인정보의 수집과 이용에 대한 동의는 구체적이고 명확한 목적을 제시한 후 받아야 합니다.",
    metadata={"source": "법률"},
)

document_8 = Document(
    page_content="개인정보 보호법은 개인정보의 수집, 사용, 제공, 관리, 보호 등에 관한 사항을 규정합니다.",
    metadata={"source": "법률"},
)

document_9 = Document(
    page_content="개인정보 보호책임자는 개인정보 처리방침의 수립과 공개, 개인정보 보호 교육 등을 수행합니다.",
    metadata={"source": "법률"},
)

document_10 = Document(
    page_content="개인정보의 수집과 이용에 대한 동의는 이용자가 쉽게 인지하고 선택할 수 있도록 명확하게 표시해야 합니다.",
    metadata={"source": "법률"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

elastic_vector_search.add_documents(documents=documents, ids=uuids)

['4192d0d1-86a7-4d76-8aff-b54c0a5a1d21',
 '5d0bc587-66f5-4126-afba-19bce252992e',
 'd5281930-da46-4893-b152-efa635ba8da9',
 'cede6be0-bdbb-44b7-969c-3c5d43d6abd4',
 '4164b984-7889-40f5-bbad-ec3802edae3d',
 '8dd71e17-cba0-4b5e-9eef-05f9fd870b30',
 '4725733b-733b-4043-b1fd-82c1f80a529e',
 '1f901ebe-d10a-4350-b758-249304a6d47d',
 '84c2d457-73b0-4adf-9532-ae41d78252d5',
 '745acdcf-564b-43e1-9554-aacd643c3dc0']

In [25]:
results = elastic_vector_search.similarity_search(
    query="개인정보란?",
    k=2,
    filter=[{"term": {"metadata.source.keyword": "법률"}}],
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* 개인정보 보호법은 개인정보의 수집, 사용, 제공, 관리, 보호 등에 관한 사항을 규정합니다. [{'source': '법률'}]
* 개인정보 보호법에 따라 개인정보 수집 시에는 반드시 이용자의 동의를 받아야 합니다. [{'source': '법률'}]


In [32]:
retriever = elastic_vector_search.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.2}
)
retriever.invoke("개인정보란?")

[Document(metadata={'source': '법률'}, page_content='개인정보 보호법은 개인정보의 수집, 사용, 제공, 관리, 보호 등에 관한 사항을 규정합니다.'),
 Document(metadata={'source': '법률'}, page_content='개인정보 보호법에 따라 개인정보 수집 시에는 반드시 이용자의 동의를 받아야 합니다.'),
 Document(metadata={'source': '법률'}, page_content='개인정보의 수집과 이용 목적이 달성되면 지체 없이 해당 개인정보를 파기해야 합니다.'),
 Document(metadata={'source': '법률'}, page_content='개인정보의 안전성 확보를 위해 암호화, 접근제한, 접속이력 관리 등의 기술적 조치를 취해야 합니다.')]

In [27]:
results = elastic_vector_search.similarity_search_with_score(
    query="Will it be hot tomorrow",
    k=1,
    filter=[{"term": {"metadata.source.keyword": "법률"}}],
)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.871169] 개인정보의 수집과 이용 목적이 달성되면 지체 없이 해당 개인정보를 파기해야 합니다. [{'source': '법률'}]
